In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import os
import time
import regex as re

## Setando Programação Paralela

In [2]:
MAX_WORKERS = os.cpu_count()
print(MAX_WORKERS)

16


## Buscar Links de todos os Produtos

In [55]:
def search_links(categoria, page,product_links):
    url = f'https://www.agillemed.com.br/loja/catalogo.php?loja=845890&categoria={categoria}&pg={page}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all("a", class_="product-info")
        urls = [link.get('href') for link in links] 
        for url in urls:
            if url not in product_links:
                product_links.append(url)
                print(page)
                print(len(product_links))
        return ''
    else:
        return None

#medicamento, medicamento especial, genericos, saude-e-bem-estar, beleza-higiene, mamae-e-bebe
#
categorias = ['115','29','105','31','77','51']
product_links = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for categoria in categorias:
        url = f'https://www.agillemed.com.br/loja/catalogo.php?loja=845890&categoria={categoria}&pg=1'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            html_content = response.content
            soup = BeautifulSoup(html_content, 'html.parser')
            div_resultados = soup.find('div', class_='results')
            texto = div_resultados.get_text()
            padrao_paginas = r'(\d+) páginas'
            resultado = re.search(padrao_paginas, texto)
            numero_paginas = int(resultado.group(1))
            page = 1
        while page < numero_paginas:
            urls = executor.submit(search_links,categoria,page,product_links).result()
            if urls is not None:
                page += 1
                
        # Tentativa com próxima página
            else: 
                page += 1
                urls = executor.submit(search_links,categoria,page,product_links).result()
                if urls is not None:
                    pass
                else:
                    break

1
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
1
22
1
23
1
24
1
25
1
26
1
27
1
28
1
29
1
30
1
31
1
32
1
33
1
34
1
35
1
36
1
37
1
38
2
39
2
40
2
41
2
42
2
43
2
44
2
45
2
46
2
47
2
48
2
49
2
50
2
51
2
52
2
53
2
54
2
55
2
56
2
57
2
58
2
59
2
60
2
61
2
62
2
63
2
64
2
65
2
66
2
67
2
68
2
69
2
70
2
71
3
72
3
73
3
74
3
75
3
76
3
77
3
78
3
79
3
80
3
81
3
82
3
83
3
84
3
85
3
86
3
87
3
88
3
89
3
90
3
91
3
92
3
93
3
94
3
95
3
96
3
97
3
98
3
99
3
100
3
101
3
102
3
103
3
104
4
105
4
106
4
107
4
108
4
109
4
110
4
111
4
112
4
113
4
114
4
115
4
116
4
117
4
118
4
119
4
120
4
121
4
122
4
123
4
124
4
125
4
126
4
127
4
128
4
129
4
130
4
131
4
132
4
133
4
134
4
135
4
136
4
137
5
138
5
139
5
140
5
141
5
142
5
143
5
144
5
145
5
146
5
147
5
148
5
149
5
150
5
151
5
152
5
153
5
154
5
155
5
156
5
157
5
158
5
159
5
160
5
161
5
162
5
163
5
164
5
165
5
166
5
167
5
168
5
169
5
170
6
171
6
172
6
173
6
174
6
175
6
176
6
177
6
178
6
179
6
180
6
181
6
182
6
183
6
184
6
18

In [56]:
len(product_links)

531

In [57]:
df = pd.DataFrame({'Links': product_links})
df.to_csv('links.csv')


## Buscando dados em cada item

In [85]:
def search_ean(soup):
    try:
        codigo_de_barras_tag = soup.find("td", text="Código de barras")
        ean = codigo_de_barras_tag.find_next_sibling("td").text.strip()
        return ean
    except:
        return None

def search_produto(soup):
    try:
        nome_produto_tag = soup.find("h1", class_="product-name")
        nome_produto = nome_produto_tag.text.strip()
        return nome_produto
    except:
        return None

def search_brand(soup):
    try:
        marca_div = soup.find("div", class_="product-brand")
        marca = marca_div.text.strip().split(":")[-1].strip()
        return marca
    except:
        return None
    
def search_farmacia(soup):
    try:
        return 'Agille'
    except:
        return None
    
def search_preco_com_desconto(soup):
    try:
        price_span = soup.find('span', id='variacaoPreco')
        price = price_span.get_text()
        price_float = float(price.replace(',', '.'))
        return price_float
    except:
        return None
    
def search_preco_sem_desconto(soup):
    try:
        precode_div = soup.find('div', class_='precode')
        price_text = precode_div.get_text()
        preco_sem_desconto = float(price_text.split()[2].replace(',', '.'))

        return preco_sem_desconto
    except:
        return None

def search_desconto(soup,preco_com_desconto,preco_sem_desconto):
    try:
        desconto = 1.0 - (preco_com_desconto / preco_sem_desconto)
        return desconto
    except:
        return 0.0
    
def search_descricao(soup):
    try:
        descricao_div = soup.find("div", class_="tabs-content")
        descricao = descricao_div.text.strip()
        return descricao
    except:
        return None

def buscar_info_products(url,ean,produto,marca,farmacia,preco_com_desconto,preco_sem_desconto,desconto,descricao,i):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url,headers=headers)
    print(i)
    
    if response.status_code == 200:
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        ean.append(search_ean(soup))
        produto.append(search_produto(soup))
        marca.append(search_brand(soup))
        farmacia.append(search_farmacia(soup))
        descricao.append(search_descricao(soup))
        preco_com_desconto.append(search_preco_com_desconto(soup))
        preco_sem_desconto.append(search_preco_sem_desconto(soup))
        desconto.append(search_desconto(soup,search_preco_com_desconto(soup),search_preco_sem_desconto(soup)))
    else:
        ean.append(None)
        produto.append(None)
        marca.append(None)
        farmacia.append(None)
        preco_com_desconto.append(None)
        preco_sem_desconto.append(None)
        desconto.append(None)
        descricao.append(None)



In [86]:
ean = []
produto = []
marca = []
farmacia = []
preco_com_desconto = []
preco_sem_desconto = []
desconto = []
descricao = []
i = 1
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submeter cada busca em paralelo
    for url in product_links:
        executor.submit(buscar_info_products, url, ean, produto, marca, farmacia, preco_com_desconto, preco_sem_desconto, desconto, descricao,i)
        i += 1

13
3
5
11
2
1
15
10
16


C:\Users\Marcelo\AppData\Local\Temp\ipykernel_12380\25438734.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  codigo_de_barras_tag = soup.find("td", text="Código de barras")


8
4
6
14
7
9
12
19
20
18
25
21
24
17
31
22
28
23
30
27
33
29
26
34
35
39
37
32
38
4041
36

44
45
43
50
51
49
52
47
42
48
56
54
55
46
57
53
58
59
60
6162
63

69
65
64
71
66
68
72
70
75
74
67
78
76
73
77
79
81
80
82
87
88
8590

91
92
84
89
95
94
83
93
97
96
86
98
99
101
100
102
103
111
104
107
112
113106

117
114
115
116
109
105
110
120
119
108
118
121
122
123
125
124
126
129
128
130
127
131
133
139
137
136138
141
142

144
143
145
132
135
134
140
151
149
150
148
153
147
146
156
154
155
152
157
158
162
161
160
165
164
159
169
167171

172
168
163
170
178
179
173
175
177
176
182
174
180
183
184
181
166
189
186
188
190
185
187
191
196
194
197
198
195
193
192
200
202
199
201
203
205
208
204
206
209
207210
211

215
216
218
213
214212

219
217
224
220
225
223
221
230
229
228
222226

227
235
239
236
237
231234

232
243
242
233
240238

245
244
246
247
241
251
252
248
250
249253

256
255
260
262
258
259
254
261
257
263
266
265
267264

268
269
272
271
270
273276

274
277
279
278
282
281
275
283
280

In [87]:
print(len(ean))
print(len(produto))
print(len(marca))
print(len(farmacia))
print(len(preco_com_desconto))
print(len(preco_sem_desconto))
print(len(desconto))
print(len(descricao))

531
531
531
531
531
531
531
531


In [88]:
df = pd.DataFrame({
    'ean': ean,
    'produto': produto,
    'marca': marca,
    'farmacia': farmacia,
    'preco com desconto': preco_com_desconto,
    'preco sem desconto': preco_sem_desconto,
    '% desconto': desconto,
    'descricao': descricao
})

In [94]:
df.head(60)

,ean,produto,marca,farmacia,preco com desconto,preco sem desconto,% desconto,descricao
0,7896658029028,Leucogen 80mg 30 Cápsulas,Ache,Agille,159.87,NaN,0.000000,Descrição Geral\n \n\n\nIndicaç...
1,7895296436014,Aciclovir 200mg 30 Comprimidos Genérico Nova Q...,Nova Química Genérico,Agille,NaN,NaN,0.000000,Descrição Geral\n \n\n\nAciclov...
2,7896512909732,Phebo Amazonian Sabonete de Glicerina Granado 90g,Granado,Agille,7.06,NaN,0.000000,Descrição Geral\n \n\n\nPhebo A...
3,7897572006485,Temodal Sachet 180mg 5 Cápsulas,Merck Sharp e Dohme,Agille,NaN,NaN,0.000000,Descrição Geral\n \n\n\nIndicaç...
4,7898932797302,"Letrozol 2,5 mg com 30 comprimidos",Accord Genérico,Agille,16.50,NaN,0.000000,Descrição Geral\n \n\n\n\nTrata...
5,7898565160085,Imatinibe 400mg 30 Comprimidos Genérico Eurofarma,Eurofarma Genérico,Agille,NaN,NaN,0.000000,Descrição Geral\n \n\n\nIndicaç...
6,7891317483777,"Letrozol 2,5mg 30 Comprimidos Genérico Natco F...",Natco Farma,Agille,20.00,NaN,0.000000,Descrição Geral\n \n\n\nIndicaç...
7,789.66764.3089,"Heparinox 60mg 0,6ml sol inj com 2 seringas pr...",Cristália,Agille,NaN,NaN,0.602000,Descrição Geral\n \n\n\nHeparin...
8,7896676439069,(Anastrozol )Cermaz 1mg 30 comprimidos,Cristália,Agille,19.90,50.00,0.000000,Descrição Geral\n \n\n\nIndicaç...
9,8901148240056,Abiraterona 250mg 120 Comprimidos Genérico Dr....,Dr. Reddy's Genérico,Agille,NaN,NaN,0.000000,Descrição Geral\n \n\n\nIndicaç...


In [ ]:
df.to_csv('info_produtos_agille.csv')